In [ ]:
import numpy as np 
import pandas as pd 
import string
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [ ]:
df_train = pd.read_csv('../input/nlp-getting-started/train.csv')
df_train

In [ ]:
df_test = pd.read_csv('../input/nlp-getting-started/test.csv')
df_test

In [ ]:
sample_submission = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
sample_submission

### EDA :

In [ ]:
df_train.shape   # There are 7613 rows and 5 columns

In [ ]:
df_train.text.isnull().values.any()   # There is no null value in text column

In [ ]:
df_train.location.value_counts()

In [ ]:
df_train.keyword.value_counts()

In [ ]:
df_train.text.describe()

In [ ]:
df_train.keyword.describe()

In [ ]:
df_train.location.describe()

### Dropping columns :

In [ ]:
df_train.drop(['location' , 'keyword'] , axis = 1 , inplace = True)

In [ ]:
df_train

In [ ]:
df_train.text[:3]

### Cleaning the data :

In [ ]:
df_train.columns

In [ ]:
df_train['text']=df_train['text'].str.replace('.' , '')
df_train['text']=df_train['text'].str.replace(',' , '')
df_train['text']=df_train['text'].str.replace('&' , '')
df_train['text']=df_train['text'].str.lower()

### Removing punctuations :

In [ ]:
string.punctuation

In [ ]:
def remove_punctuation(text):
    without_punct="".join([i for i in text if i not in string.punctuation])
    return without_punct

In [ ]:
df_train['text']= df_train['text'].apply(lambda x:remove_punctuation(x))

### Tokenization :

In [ ]:
def tokenize(string):
    '''
    Tokenizes the string to a list of words
    '''
    word_tokens = string.split()
    return word_tokens

In [ ]:
df_train['text']= df_train['text'].apply(lambda x: tokenize(x))

In [ ]:
df_train.head()

### Removing stop words :

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

In [ ]:
df_train['text']= df_train['text'].apply(lambda x:remove_stopwords(x))

### Stemming :

In [ ]:
porter_stemmer = PorterStemmer()

In [ ]:
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text
df_train['text']=df_train['text'].apply(lambda x: stemming(x))

### Lematization :

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

In [ ]:
nltk.download('wordnet')

In [ ]:
df_train['text']=df_train['text'].apply(lambda x: lemmatizer(x))

### Text vectorization :

In [ ]:
## Converting the tokens back to strings to feed it into Count Vectorizer

df_train['text_strings'] = df_train['text'].apply(lambda x: ' '.join([str(word) for word in x]))

In [ ]:
# df_train

In [ ]:
df_train['text_strings'].head()


## Text vectorization :

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_train['text_strings'])


In [ ]:
x_train = X.toarray()

In [ ]:
x_train = np.array(x_train)

In [ ]:
x_train.shape

In [ ]:
y_train = df_train['target']

In [ ]:
y_train.shape

### Fitting model: (Logistic regression)

In [ ]:
clf = LogisticRegression(random_state=42)
clf.fit(x_train,y_train)

In [ ]:
pred = clf.predict(x_train)

In [ ]:
accuracy_score(y_train, pred)

In [ ]:
classification_report(y_train, pred)


### Fitting model : (Random Forest)

In [ ]:
clf2 = RandomForestClassifier(n_estimators=100, random_state=42, max_depth = 700)  #500
clf2.fit(x_train,y_train)

In [ ]:
pred2 = clf2.predict(x_train)

In [ ]:
accuracy_score(y_train, pred2)

### Applying on test data :

In [ ]:
df_test.fillna('',inplace=True)
df_test.drop(columns=['id','keyword','location'],inplace=True)

In [ ]:
df_test['text']= df_test['text'].apply(lambda x:remove_punctuation(x))
df_test['text']= df_test['text'].apply(lambda x: tokenize(x))
df_test['text']= df_test['text'].apply(lambda x:remove_stopwords(x))
df_test['text']= df_test['text'].apply(lambda x: stemming(x))

In [ ]:
df_test['text_strings'] = df_test['text'].apply(lambda x: ' '.join([str(elem) for elem in x]))

In [ ]:
x_test = vectorizer.transform(df_test['text_strings'])
x_test = x_test.toarray()


In [ ]:
x_test = np.array(x_test)

In [ ]:
y_test_pred = clf2.predict(x_test)

In [ ]:
y_test_pred

### Saving output to csv :

In [ ]:
submission = pd.read_csv('../input/nlp-getting-started/test.csv')

In [ ]:
submission['target'] = y_test_pred

In [ ]:
submission.head()

In [ ]:
final_submission = submission[['id','target']]

In [ ]:
final_submission.to_csv('final_submission.csv', index = False)